# Machine Learning pour prédire l'état de santé d'un arbre

Dans ce notebook, vous trouvez notre code d'entraînement et validation automatique d'algorithmes de Machine Learning pour prédire l'état de santé d'un arbre(C0 = emplacement vide ; C1 = arbre sain, de bonne vigueur ; C2 = arbre présentant des lésions sans gravité ; C3 = arbre présentant des lésions importantes ; C4 = arbre présentant des lésions importantes et évolutives ; C5 = arbre présentant des lésions irrémédiables nécessitant un abattage ; C6 = souche) à partir du jeu de données *Arbres urbains*.

**dataset:** https://www.data.gouv.fr/fr/datasets/arbres-urbains/<br>
**tâche:** Classification (Classification multi-classe)<br>
**target variable:** *classification_diagnostic*

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

On importe le dataset à partir de son url.

In [2]:
url = 'https://www.data.gouv.fr/fr/datasets/r/96f4164d-956d-4c1c-b161-68724eb0ccdc'
data = pd.read_csv(url)

## 2. Preprocessing

Grâce à `Pandas Profiling`(https://giuliasantarsieri.github.io/open_ML/profilings/arbres_urbains.html), on a pu identifier les colonnes inutiles et non supportées. On les soustrait du dataset.

In [3]:
data = data.drop(columns=['ID_ARBRE','commune','controle','champignon_collet', 'insecte_collet','champignon_collet', 'insecte_collet','champignon_tronc', 'insecte_tronc',
       'fissure_tronc','ecorce_incluse_houppier','type_delai_1', 'delai_annee_programmation',
       'type_delai_2', 'delai_preconisation_2', 'delai_saison_programmation_2',
       'delai_annee_programmation_2', 'reference_photo'])

On fait le choix d'éliminer les variables ayant plus de 80% de valeurs manquantes et de faire de l'imputation des valeurs manquantes pour les autres colonnes

In [6]:
data = data.drop(columns=[col for col in data.columns if data[col].isna().sum()/len(data)>0.8])

imp = SimpleImputer(strategy="most_frequent")
data[['cote_voirie', 'prescription_1', 'prescription_2']] = imp.fit_transform(data[['cote_voirie', 'prescription_1', 'prescription_2']])

## 3. Train/test splitting

On définit la variable réponse et les variables explicatives. On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [7]:
y = data['classification_diagnostic'].values
X = data.drop(columns=['classification_diagnostic'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.


In [ ]:
automl = AutoML(total_time_limit=5*60,mode='Explain',random_state=42)
#fit model
automl.fit(X_train,y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()

### Preprocessing fait par AutoML

Il n'est pas évident de comprendre, à partir du rapport html qui est généré, quelles tâches de preprocessing sont faites dans la **pipeline de AutoML** qui permet d' aboutir aux résultats observés. <br>
Pour mieux comprendre les résultats, on peut s'intéresser au fichier `framework.json` de chaque modèle. Par exemple, on peut consulter ce fichier pour le modèle `Xgboost` ici:
https://github.com/etalab-ia/DGML/blob/main/docs/automodels/c763b24a-a0fe-4e77-9586-3d5453c631cd/4_Default_Xgboost/framework.json<br>
On observe:
1. L'imputation des **valeurs manquantes** est faite en remplaçant par la valeur médiane
2. L'**encoding** des variables catégorielles est fait en transformant le nom de chaque catégorie en un entier

Ces informations sont disponibles pour tous les algorithmes.